# Projet - Apprentissage Profond - Implémentation (Partie Deep Network)

**EL KAAKOUR Ahmad & Matthieu RANDRIANTSOA**

> Ce notebook contient toutes les fonctions d'implémentation de nos modèles. Il est conçu de sorte à s'exécuter sequentiellement.

In [1]:
import gzip, numpy, torch
import matplotlib.pyplot as plt
from pathlib import Path
import itertools
import json
import os
import pandas as pd
import seaborn as sns

In [22]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Partie 3 : Deep Network

## Chargement des données

In [7]:
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('data/mnist.pkl.gz'), weights_only=False)

## Préparation des données

In [8]:
training_dataset = torch.utils.data.TensorDataset(data_train, label_train)
test_dataset = torch.utils.data.TensorDataset(data_test, label_test)

# Diviser le jeu de données en sous-ensembles
generator = torch.Generator().manual_seed(42) # Permet de reproduire le même découpement
train_subset, val_subset = torch.utils.data.random_split(training_dataset, [0.8, 0.2], generator=generator) # on divise le jeu de données en 80% pour l'entraînement et 20% pour la validation

## Création du modèle de réseau de neurones

### Variante à trois couches cachées

In [3]:
class DeepNeuralNetwork(torch.nn.Module):
    def __init__(self,nb_neurons_1:int, nb_neurons_2:int, nb_neurons_3:int):
        super(DeepNeuralNetwork,self).__init__()

        # première couche cachée
        self.hidden_layer_1 = torch.nn.Linear(28*28,nb_neurons_1)
        # deuxième couche cachée
        self.hidden_layer_2 = torch.nn.Linear(nb_neurons_1, nb_neurons_2)
        # troisième couche cachée
        self.hidden_layer_3 = torch.nn.Linear(nb_neurons_2, nb_neurons_3)
        # couche de sortie
        self.output_layer = torch.nn.Linear(nb_neurons_3,  10)

    def forward(self,x:torch.Tensor):
        x = self.hidden_layer_1(x)
        x = torch.nn.functional.relu(x)
        x = self.hidden_layer_2(x)
        x = torch.nn.functional.relu(x)
        x = self.hidden_layer_3(x)
        x = torch.nn.functional.relu(x)
        x = self.output_layer(x)
        return x

###Variantes à deux couches cachées

In [ ]:
class DeepNeuralNetwork_2(torch.nn.Module):
    def __init__(self,nb_neurons_1:int, nb_neurons_2:int):
        super(DeepNeuralNetwork,self).__init__()

        # première couche cachée
        self.hidden_layer_1 = torch.nn.Linear(28*28,nb_neurons_1)
        # deuxième couche cachée
        self.hidden_layer_2 = torch.nn.Linear(nb_neurons_1, nb_neurons_2)
        #couche de sortie
        self.output_layer = torch.nn.Linear(nb_neurons_2,  10)

    def forward(self,x:torch.Tensor):
        #passage en avant
        x = self.hidden_layer_1(x)
        x = torch.nn.functional.relu(x)
        x = self.hidden_layer_2(x)
        x = torch.nn.functional.relu(x)
        x = self.output_layer(x)
        return x

## Fonctions d'entrainement

In [ ]:
def train_and_evaluate_2(_model, _training_loader, _validation_loader, _params, device='cpu'):
    # Extraire les hyperparamètres depuis params
    _nb_epochs = _params.get('nb_epochs', 10)
    _loss_func = _params.get('loss_func', torch.nn.CrossEntropyLoss())
    _optim = _params.get('optimizer', torch.optim.SGD(_model.parameters(), lr=_params.get('learning_rate', 0.001)))

    # Déplacer le modèle vers le périphérique spécifié (GPU si disponible)
    _model.to(device)

    # Initialiser des listes pour suivre les pertes et les précisions afin d'analyser les performances
    training_losses = []
    validation_losses = []
    validation_accuracies = []

    # Boucle d'entraînement
    for epoch in range(_nb_epochs):
        _model.train()  # Mettre le modèle en mode entraînement
        epoch_training_loss = 0.0

        # Étape d'entraînement
        for x, t in _training_loader:
            x, t = x.to(device), t.to(device)  # Déplacer les données vers le périphérique

            # Passer en avant (forward pass)
            y = _model(x)

            # Calculer la perte
            _loss = _loss_func(y, t)
            epoch_training_loss += _loss.item()

            # Rétropropagation et optimisation
            _optim.zero_grad()  # Réinitialiser les gradients
            _loss.backward()  # Calculer les gradients
            _optim.step()  # Mettre à jour les poids du modèle

        # Perte moyenne d'entraînement pour l'époque
        avg_training_loss = epoch_training_loss / len(_training_loader)
        training_losses.append(avg_training_loss)

        # Étape de validation
        _model.eval()  # Mettre le modèle en mode évaluation
        epoch_val_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        # Désactiver le calcul des gradients pendant l'évaluation
        with torch.inference_mode():
            for data, target in _validation_loader:  # Utiliser validation_loader pour la validation
                data, target = data.to(device), target.to(device)  # Déplacer les données vers le périphérique

                # Passer en avant (forward pass)
                outputs = _model(data)

                # Calculer la perte
                _loss = _loss_func(outputs, target)
                epoch_val_loss += _loss.item()

                # Calculer la précision
                predicted_labels = torch.argmax(outputs, dim=1)
                true_labels = torch.argmax(target, dim=1)
                correct_predictions += (predicted_labels == true_labels).sum().item()
                total_predictions += target.size(0)

        # Perte moyenne de validation et précision pour l'époque
        avg_val_loss = epoch_val_loss / len(_validation_loader)
        validation_losses.append(avg_val_loss)

        accuracy = correct_predictions / total_predictions
        validation_accuracies.append(accuracy)

        # Afficher les métriques pour l'époque en cours
        print(f"Époque {epoch + 1}/{_nb_epochs}, Validation loss: {avg_val_loss:.4f}, Validation accuracy: {accuracy:.4f}")

    # Calcul de la précision finale sur le jeu de test
    final_correct_predictions = 0
    final_total_predictions = 0

    _model.eval()  # Mettre le modèle en mode évaluation
    with torch.inference_mode():
        for data, target in _validation_loader:  # Utiliser test_loader pour la phase finale de test
            data, target = data.to(device), target.to(device)
            outputs = _model(data)
            predicted_labels = torch.argmax(outputs, dim=1)
            true_labels = torch.argmax(target, dim=1)
            final_correct_predictions += (predicted_labels == true_labels).sum().item()
            final_total_predictions += target.size(0)

    # Calculer la précision finale sur le jeu de test
    final_test_accuracy = final_correct_predictions / final_total_predictions
    print(f"Précision finale: {final_test_accuracy:.4f}")

    # Retourner les métriques de performance pour analyse ultérieure
    return {
        'hyperparameters': _params,
        'training_losses': training_losses,
        'validation_losses': validation_losses,
        'validation_accuracies': validation_accuracies,
        'final_validation_accuracy': final_test_accuracy
    }


In [ ]:
def setup_deep_network(_params, device='cpu'):
    """
      Prépare les données et intialise l'entrainement du modèle
    """
    _batch_size = _params.get('batch_size', 8)
    _nb_neurons_1 = _params.get('nb_neurons_1', 10)
    _nb_neurons_2 = _params.get('nb_neurons_2', 10)
    _nb_neurons_3 = _params.get('nb_neurons_3', 10)
    _nb_epochs =_params.get('nb_epochs', 10)

    # Create Loader
    train_loader = torch.utils.data.DataLoader(train_subset, batch_size=_batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_subset, batch_size=_batch_size, shuffle=False)

    # Model
    _model = DeepNeuralNetwork(_nb_neurons_1, _nb_neurons_2, _nb_neurons_3)

    # Call training function
    print(f"Training the model : {_params.get('learning_rate', 0.001), _batch_size, _nb_epochs, _nb_neurons_1, _nb_neurons_2, _nb_neurons_3}")
    return train_and_evaluate_2(_model, train_loader, val_loader, _params, device)

## Définir les hyperparamètres et définir une plage de valeurs à explorer

In [ ]:
#configurations des hyperparamètres fixes et des neurones à faire varier
param_grid = {
    "nb_neurones_1_range" :  [10, 20, 32, 64, 128, 256, 512, 1024],
    "nb_neurones_2_range" :  [10, 20, 32, 64, 128, 256, 512, 1024],
    "nb_neurones_3_range" :  [10, 20, 32, 64, 128, 256, 512, 1024]
}

# Combinaisons d'hyperparamètres avec les valeurs fixes pour les autres paramètres
learning_rate = 0.01
batch_size = 8
nb_epochs = 10

param_combinations = list(itertools.product(param_grid['nb_neurones_1_range'],
                                            param_grid['nb_neurones_2_range'],
                                            param_grid['nb_neurones_3_range'])
                        )


## Entrainement des différents modèles

In [ ]:
# Chemin vers le fichier de sortie
file_path = "data/model_metrics_1809024_bis_follow_up.json"

# Vérifie si le fichier existe déjà pour savoir si c'est le premier ajout
first_write = not os.path.exists(file_path)

res = []

# Ouvre le fichier en mode ajout (append) pour écrire progressivement les résultats
with open(file_path, "a") as json_file:

    # Si c'est la première écriture, on ouvre un tableau JSON avec un crochet ouvrant
    if first_write:
        json_file.write("[\n")  # Début de l'array JSON

    # Boucle sur toutes les combinaisons d'hyperparamètres
    for i, (nb_neurons_1, nb_neurons_2, nb_neurons_3) in enumerate(param_combinations):

        # Calcule les résultats pour les hyperparamètres actuels
        result = setup_deep_network(_params={
            "batch_size": batch_size,
            "nb_epochs": nb_epochs,
            "learning_rate": learning_rate,
            "nb_neurons_1": nb_neurons_1,
            "nb_neurons_2": nb_neurons_2,
            "nb_neurons_3": nb_neurons_3
        })
        #
        res.append(result)

        # Écrit les résultats au format JSON dans le fichier
        json.dump(result, json_file)

        # Ajoute une virgule et un saut de ligne après chaque entrée, sauf la dernière
        if i < len(param_combinations) - 1:
            json_file.write(",\n")

    # Si c'est la première écriture, on ferme le tableau avec un crochet fermant après la boucle
    if first_write:
        json_file.write("\n]")

## Amélioration des cinqs meilleurs modèles
> On se base sur les observations présenté dans la partie 'Analyse des modèles'.

In [ ]:
#Re-entrainement des 5 meilleurs modèles
best_model_configurations = [
    {"nb_neurons_1": 1024, "nb_neurons_2": 10, "nb_neurons_3": 1024},
    {"nb_neurons_1": 1024, "nb_neurons_2": 20, "nb_neurons_3": 1024},
    {"nb_neurons_1": 512, "nb_neurons_2": 10, "nb_neurons_3": 1024},
    {"nb_neurons_1": 256, "nb_neurons_2": 10, "nb_neurons_3": 1024},
    {"nb_neurons_1": 1024, "nb_neurons_2": 10, "nb_neurons_3": 512}
]

#hyperparamètres fixes
learning_rate = 0.01
batch_size = 32
nb_epochs = 50

# Chemin vers le fichier de sortie
file_path = "data/model_deep_reentrainement_top5.json"

# Vérifie si le fichier existe déjà pour savoir si c'est le premier ajout
first_write = not os.path.exists(file_path)

res = []

# Ouvre le fichier en mode ajout (append) pour écrire progressivement les résultats
with open(file_path, "a") as json_file:

    # Si c'est la première écriture, on ouvre un tableau JSON avec un crochet ouvrant
    if first_write:
        json_file.write("[\n")  # Début de l'array JSON
        
    # Boucle sur les 5 meilleurs modèles
    for config in best_model_configurations:
        nb_neurons_1 = config["nb_neurons_1"]
        nb_neurons_2 = config["nb_neurons_2"]
        nb_neurons_3 = config["nb_neurons_3"]
        
        print(f"Réentraînement du modèle avec nb_neurons_1 :{nb_neurons_1} , nb_neurons_2 : {nb_neurons_2} et nb_neurons_3: {nb_neurons_3}")
        
        # Calcul des résultats pour les hyperparamètres actuels
        result = setup_deep_network(_params={
            "batch_size": batch_size,
            "nb_epochs": nb_epochs,
            "learning_rate": learning_rate,
            "nb_neurons_1": nb_neurons_1,
            "nb_neurons_2": nb_neurons_2,
            "nb_neurons_3": nb_neurons_3
        })
        
        res.append(result)

        # Écrit les résultats au format JSON dans le fichier
        json.dump(result, json_file)

        # Ajoute une virgule et un saut de ligne après chaque entrée, sauf la dernière
        json_file.write(",\n")
        

    # Si c'est la première écriture, on ferme le tableau avec un crochet fermant après la boucle
    if first_write:
        json_file.write("\n]")

# Analyse des modèles

## Deux couches cachées

In [3]:
#on transforme les données du fichier en dataframe
with open('data/model_metrics_DL_2L.json') as json_file:
    data = json.load(json_file)

#on crée une liste vide pour stocker les données structurées
structured_data = []

# on parcourt les données du fichier
for entry in data:
    # on extrait les hypermatères
    hyperparameters = entry['hyperparameters']
    batch_size = hyperparameters['batch_size']
    nb_epochs = hyperparameters['nb_epochs']
    learning_rate = hyperparameters['learning_rate']
    nb_neurons_1 = hyperparameters['nb_neurons_1']
    nb_neurons_2 = hyperparameters['nb_neurons_2']
    # nb_neurons_3 = hyperparameters['nb_neurons_3']
    training_losses = entry['training_losses']
    validation_losses = entry['validation_losses']
    validation_accuracies = entry['validation_accuracies']
    final_validation_accuracy = entry['final_validation_accuracy']

    # on crée un dictionnaire avec les données structurées
    structured_data.append({
        'batch_size': batch_size,
        'nb_epochs': nb_epochs,
        'learning_rate': learning_rate,
        'nb_neurons_1': nb_neurons_1,
        'nb_neurons_2': nb_neurons_2,
        # 'nb_neurons_3': nb_neurons_3,
        'training_losses': training_losses,
        'validation_losses': validation_losses,
        'validation_accuracies': validation_accuracies,
        'final_validation_accuracy': final_validation_accuracy
    })

# on crée un dataframe avec les données structurées
df = pd.DataFrame(structured_data)
df.sort_values("final_validation_accuracy", ascending=False, inplace=True)

In [29]:
df[['learning_rate', 'batch_size', 'nb_epochs', 'nb_neurons_1', 'nb_neurons_2', 'final_validation_accuracy']]


,learning_rate,batch_size,nb_epochs,nb_neurons_1,nb_neurons_2,final_validation_accuracy
76,0.01,32,25,2048,128,0.975159
80,0.01,32,25,2048,2048,0.974762
79,0.01,32,25,2048,1024,0.974524
70,0.01,32,25,1024,1024,0.974444
64,0.01,32,25,1024,20,0.974444
...,...,...,...,...,...,...
4,0.01,32,25,10,128,0.939206
3,0.01,32,25,10,64,0.936746
2,0.01,32,25,10,32,0.934683
1,0.01,32,25,10,20,0.931587


## Trois couches cachées

In [14]:
#on transforme les données du fichier en dataframe
with open('data/model_metrics_DL_Top_5.json') as json_file:
    data = json.load(json_file)
    
# with open('data/model_metrics_DL_3L.json') as json_file:
#     data = json.load(json_file)

#on crée une liste vide pour stocker les données structurées
structured_data = []

# on parcourt les données du fichier
for entry in data:
    # on extrait les hypermatères
    hyperparameters = entry['hyperparameters']
    batch_size = hyperparameters['batch_size']
    nb_epochs = hyperparameters['nb_epochs']
    learning_rate = hyperparameters['learning_rate']
    nb_neurons_1 = hyperparameters['nb_neurons_1']
    nb_neurons_2 = hyperparameters['nb_neurons_2']
    nb_neurons_3 = hyperparameters['nb_neurons_3']
    training_losses = entry['training_losses']
    validation_losses = entry['validation_losses']
    validation_accuracies = entry['validation_accuracies']
    final_validation_accuracy = entry['final_validation_accuracy']

    # on crée un dictionnaire avec les données structurées
    structured_data.append({
        'batch_size': batch_size,
        'nb_epochs': nb_epochs,
        'learning_rate': learning_rate,
        'nb_neurons_1': nb_neurons_1,
        'nb_neurons_2': nb_neurons_2,
        'nb_neurons_3': nb_neurons_3,
        'training_losses': training_losses,
        'validation_losses': validation_losses,
        'validation_accuracies': validation_accuracies,
        'final_validation_accuracy': final_validation_accuracy
    })

# on crée un dataframe avec les données structurées
df_deep = pd.DataFrame(structured_data)
df_deep.sort_values("final_validation_accuracy", ascending=False, inplace=True)

In [18]:
df_deep.head(5)[['learning_rate', 'batch_size', 'nb_epochs', 'nb_neurons_1', 'nb_neurons_2', 'final_validation_accuracy']]

,learning_rate,batch_size,nb_epochs,nb_neurons_1,nb_neurons_2,final_validation_accuracy
0,0.01,32,50,1024,10,0.979921
1,0.01,32,50,1024,20,0.979841
4,0.01,32,50,1024,10,0.978810
2,0.01,32,50,512,10,0.978333
3,0.01,32,50,256,10,0.978333


# Test final

In [23]:
# Params
_nb_neurons_1 = 1024
_nb_neurons_2 = 10
_nb_neurons_3 = 1024
_learning_rate = 0.01
_batch_size = 32
_nb_epochs = 50

# Load data
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('data/mnist.pkl.gz'), weights_only=False)
training_dataset = torch.utils.data.TensorDataset(data_train, label_train)
test_dataset = torch.utils.data.TensorDataset(data_test, label_test)
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
generator = torch.Generator().manual_seed(42) # Permet de reproduire le même découpement
train_subset, val_subset = torch.utils.data.random_split(training_dataset, [0.8, 0.2], generator=generator)
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=_batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_subset, batch_size=_batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)


model = DeepNeuralNetwork(_nb_neurons_1, _nb_neurons_2, _nb_neurons_3)

#
_loss_func = torch.nn.CrossEntropyLoss()
_optimizer = torch.optim.SGD(model.parameters(), lr=_learning_rate)


# Move model to device
model.to(device)

# Training loop
for epoch in range(_nb_epochs):
    model.train()  # Set model to training mode
    for x, t in train_loader:
        x, t = x.to(device), t.to(device)  # Move data to device

        # Forward pass
        y = model(x)

        # Compute loss
        loss = _loss_func(y, t)

        # Backpropagation and optimization
        _optimizer.zero_grad()  # Reset gradients
        loss.backward()  # Compute gradients
        _optimizer.step()  # Update model weights

    # Validation step (optional)
    model.eval()  # Set model to evaluation mode
    with torch.inference_mode():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = _loss_func(outputs, target)

# save model
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "deep_neural_network.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

acc = 0.
# on lit toutes les donnéees de test
for x,t in test_loader:
  # on calcule la sortie du modèle
  y = model(x)
  # on regarde si la sortie est correcte
  acc += torch.argmax(y,1) == torch.argmax(t,1)
# on affiche le pourcentage de bonnes réponses
print(acc/data_test.shape[0])

KeyboardInterrupt: 

## Importer le modèle

In [5]:
# Params
_nb_neurons_1 = 1024
_nb_neurons_2 = 10
_nb_neurons_3 = 1024

# Load the model
model = DeepNeuralNetwork(_nb_neurons_1, _nb_neurons_2, _nb_neurons_3)
model.load_state_dict(torch.load("models/model_deep_network.pth"))

<All keys matched successfully>

In [10]:
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('data/mnist.pkl.gz'), weights_only=False)

test_dataset = torch.utils.data.TensorDataset(data_test, label_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [11]:
acc = 0.
# on lit toutes les donnéees de test
for x,t in test_loader:
  # on calcule la sortie du modèle
  y = model(x)
  # on regarde si la sortie est correcte
  acc += torch.argmax(y,1) == torch.argmax(t,1)
# on affiche le pourcentage de bonnes réponses
print(acc/data_test.shape[0])

tensor([0.9806])
